In [50]:
import os
import math
import random
import seaborn
import itertools
import numpy as np
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from statsmodels.tsa.seasonal import seasonal_decompose
from collections import namedtuple


In [635]:
df = pd.read_excel("M3C.xls", usecols="A:Z")

df_micro = df.iloc[0:146,]
df_micro = df_micro.iloc[:,6:27]

In [ ]:
# De-trend (each time series at a time)
data = pd.DataFrame(df_micro.iloc[0])
data.columns = ["value"]
year = np.arange(0, 20)
data['year'] = year
data = data.set_index('year')

# Perform seasonal decomposition
decomposition = seasonal_decompose(data['value'], model='additive', period=10)

# Access the components of the decomposition
trend = decomposition.trend
#seasonal = decomposition.seasonal
#residual = decomposition.resid
test2 = pd.DataFrame(trend).plot()
trend


In [636]:
df_train = df_micro.iloc[:,:-6]
df_test = df_micro.iloc[:, -6:]

# Standardising
scaler = StandardScaler()
df_train = scaler.fit_transform(df_train.to_numpy().reshape(-1,1))
df_train = pd.DataFrame(df_train)
MEAN = scaler.mean_
STD = scaler.scale_


In [637]:
def get_labelled_window(x, horizon=1):
  return x[:, :-horizon], x[:, -horizon]

def make_windows(x, window_size=4, horizon=1):
  window_step = np.expand_dims(np.arange(window_size+horizon), axis=0)
  window_indexes = window_step + np.expand_dims(np.arange(len(x)-(window_size+horizon-1)), axis=0).T # create 2D array of windows of window size
  windowed_array = x[window_indexes]
  windows, labels = get_labelled_window(windowed_array, horizon=horizon)
  return windows.reshape(-1,4), labels.reshape(-1,1)

In [638]:
df_test = df_test.to_numpy().reshape(-1,1)
df_test = pd.DataFrame(df_test)

In [647]:
train_x, train_y = make_windows(df_train.to_numpy(), window_size=4, horizon=1)
test2_x, test2_y = make_windows(df_test.to_numpy(), window_size=4, horizon=1)
train_x_2 = train_x + 2
train_x_2 = np.log(train_x_2)
train_y_2 = train_y + 2
train_y_2 = np.log(train_y_2)

In [640]:
# Create a function to implement a ModelCheckpoint callback
def create_model_checkpoint(model_name, save_path="model_experiments"):
  return tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(save_path, model_name),
                                            verbose=0,
                                            save_best_only=True)

In [641]:
# SMAPE
def evaluate_smape(y_true, y_pred):
    return 200 * np.mean(np.abs(y_pred - y_true) / (np.abs(y_pred) + np.abs(y_true)))

def evaluate_mdape(y_true, y_pred):
 return np.median((np.abs(np.subtract(y_true, y_pred)/ y_true))) * 100

def calculate_average_rankings(y_true, y_pred):
    num_series = len(y_pred)
    num_methods = len(y_pred[0])

    ranks = []  # to store ranks for each series

    for series_index in range(num_series):
        sape_values = [
            abs((y_true[series_index] - forecast) / y_true[series_index]) * 100
            for forecast in y_pred[series_index]
        ]
        sorted_sape = sorted(sape_values)  # sort SAPE values in ascending order
        series_ranks = [sorted_sape.index(sape) + 1 for sape in sape_values]  # assign ranks to SAPE values
        ranks.append(series_ranks)

    mean_ranks = []  # to store mean ranks for each forecasting method

    for method_index in range(num_methods):
        total_rank = sum(ranks[series_index][method_index] for series_index in range(num_series))
        mean_rank = total_rank / num_series
        mean_ranks.append(mean_rank)

    return mean_ranks

In [642]:
def evaluate_pred(y_true, y_pred):
    # Symmetric mean absolute percentage error
    smape = evaluate_smape(y_true, y_pred)
    # Median symmetric absolute percentage error
    mdape = evaluate_mdape(y_true, y_pred)
    return smape, mdape

In [643]:
def evaluate_model(y_true_set, y_pred_set):
    # Average Ranking
    avg_ranking = None
    # Percentage Better
    percentage_better = None

In [644]:
# Destandardise
def de_standardise(value):
    return value * STD + MEAN

def standardise(value):
    return (value - MEAN) / STD

In [645]:
# Hyperparameters
Combination = namedtuple("Combination", "learning_rate batch_size regularization hidden_layers")

learning_rates = np.array([0.001, 0.01, 0.1])
batch_sizes = np.array([16, 32, 64, 128, 256])
regularizations = np.array([0.001, 0.01, 0.1])
hidden_layers = np.array([2, 3, 4, 5, 6, 10])

combinations = list(itertools.starmap(Combination, itertools.product(learning_rates, batch_sizes, regularizations, hidden_layers)))

In [652]:
tf.random.set_seed(42)
eval_scores = []
tscv = TimeSeriesSplit(n_splits=24)

def cross_validation(combination, train_x=train_x, train_y=train_y, tscv=tscv):
    best_smape = float('inf')
    best_hyperparameters = {}
    hidden_neurons = np.arange(2, 9)
    smape_scores = []
    mdape_scores = []

    # Cross-Validation
    for train_index, test_index in tscv.split(train_x):
        train_x_cv, test_x_cv = train_x[train_index], train_x[test_index]
        train_y_cv, test_y_cv = train_y[train_index], train_y[test_index]
        

        # Create model with selected hyperparameters
        model_cv = tf.keras.Sequential([
            tf.keras.layers.Flatten(input_shape=(4, 1)),
        ], name="model")

        chosen_hidden_neurons = []

        for i in range(combination.hidden_layers):
            random_neuron = random.choice(hidden_neurons)
            chosen_hidden_neurons.append(random_neuron)
            model_cv.add(tf.keras.layers.Dense(random_neuron, 
                                            activation="relu", 
                                            kernel_initializer=tf.initializers.HeNormal(), 
                                            kernel_regularizer=tf.keras.regularizers.l2(combination.regularization)))
        model_cv.add(tf.keras.layers.Dense(1, activation="linear", 
                                        kernel_initializer=tf.initializers.HeNormal(), 
                                        kernel_regularizer=tf.keras.regularizers.l2(combination.regularization)))


        model_cv.compile(loss="mse",
                        optimizer=tf.keras.optimizers.Adam(learning_rate=combination.learning_rate),
                        metrics=["mse", "mae"]) # Backpropagation

        model_cv.fit(train_x_cv, train_y_cv, epochs=50, batch_size=combination.batch_size, verbose=0)
        predictions = model_cv.predict(test_x_cv)
        smape_score, mdape_score = evaluate_pred(de_standardise(test_y_cv), de_standardise(predictions))
        smape_scores.append(smape_score)
        mdape_scores.append(mdape_score)
        
    mean_smape = np.mean(smape_scores)
    mean_mdape = np.mean(mdape_scores)
    hyperparameters = {
        'learning_rate': combination.learning_rate,
        'batch_size': combination.batch_size,
        'regularization': combination.regularization,
        'hidden_neurons': chosen_hidden_neurons,
        'hidden_layers': combination.hidden_layers
    }
    print(f"Current mean SMAPE: {mean_smape}, Current hyperparameters: {hyperparameters}")
    return mean_smape, mean_mdape, hyperparameters

random_combinations = random.sample(combinations, 1)
results = map(cross_validation, random_combinations)

optimal_smape = float('inf')
optimal_mdape = float('inf')
optimal_hyperparameters = {}
for result in results:
    smape, mdape, hyperparameters = result
    if smape < optimal_smape:
        optimal_smape = smape
        optimal_mdape = mdape
        optimal_hyperparameters = hyperparameters
print("Best Hyperparameters:", optimal_hyperparameters)
print("Best SMAPE Score:", optimal_smape)
print("Best MDAPE Score:", optimal_mdape)

3/3 [==============================] - 0s 1ms/step
Current mean SMAPE: 22.07166089233959, Current hyperparameters: {'learning_rate': 0.01, 'batch_size': 16, 'regularization': 0.001, 'hidden_neurons': [3, 6], 'hidden_layers': 2}
Best Hyperparameters: {'learning_rate': 0.01, 'batch_size': 16, 'regularization': 0.001, 'hidden_neurons': [3, 6], 'hidden_layers': 2}
Best SMAPE Score: 22.07166089233959
Best MDAPE Score: 12.31710780128442


In [622]:
from bayes_opt import BayesianOptimization

def cross_validation(combination, train_x=train_x, train_y=train_y, tscv=tscv):
    best_smape = float('inf')
    best_hyperparameters = {}
    hidden_neurons = np.arange(2, 20)
    smape_scores = []
    mdape_scores = []

    # Cross-Validation
    for train_index, test_index in tscv.split(train_x):
        train_x_cv, test_x_cv = train_x[train_index], train_x[test_index]
        train_y_cv, test_y_cv = train_y[train_index], train_y[test_index]
        

        # Create model with selected hyperparameters
        model_cv = tf.keras.Sequential([
            tf.keras.layers.Flatten(input_shape=(4, 1)),
        ], name="model")

        for i in range(combination.hidden_layers):
            model_cv.add(tf.keras.layers.Dense(combination.hidden_neurons, 
                                            activation="relu", 
                                            kernel_initializer=tf.initializers.HeNormal(), 
                                            kernel_regularizer=tf.keras.regularizers.l2(combination.regularization)))
        model_cv.add(tf.keras.layers.Dense(1, activation="linear", 
                                        kernel_initializer=tf.initializers.HeNormal(), 
                                        kernel_regularizer=tf.keras.regularizers.l2(combination.regularization)))


        model_cv.compile(loss="mse",
                        optimizer=tf.keras.optimizers.Adam(learning_rate=combination.learning_rate),
                        metrics=["mse", "mae"]) # Backpropagation

        model_cv.fit(train_x_cv, train_y_cv, epochs=50, batch_size=combination.batch_size, verbose=0)
        predictions = model_cv.predict(test_x_cv)
        smape_score, mdape_score = evaluate_pred(de_standardise(test_y_cv), de_standardise(predictions))
        smape_scores.append(smape_score)
        mdape_scores.append(mdape_score)
        
        mean_smape = np.mean(smape_scores)
        mean_mdape = np.mean(mdape_scores)
        hyperparameters = {
            'learning_rate': combination.learning_rate,
            'batch_size': combination.batch_size,
            'regularization': combination.regularization,
            'hidden_neurons': combination.hidden_neurons,
            'hidden_layers': combination.hidden_layers
        }
    return mean_smape, mean_mdape, hyperparameters

# Hyperparameters
Combination = namedtuple("Combination", "learning_rate batch_size regularization hidden_layers hidden_neurons")

def objective(learning_rate, batch_size, regularization, hidden_layers, hidden_neurons):
    # Convert hyperparameters to appropriate types
    learning_rate = float(learning_rate)
    batch_size = int(batch_size)
    regularization = float(regularization)
    hidden_layers = int(hidden_layers)
    hidden_neurons = int(hidden_neurons)

    # Define the combination object with the selected hyperparameters
    combination = Combination(learning_rate, batch_size, regularization, hidden_layers, hidden_neurons)

    # Call the cross_validation function with the selected hyperparameters
    smape, mdape, _ = cross_validation(combination)

    # Return the negative SMAPE score (Bayesian optimization maximizes the objective function)
    return -smape

pbounds = {'learning_rate': (0.0001, 0.1),
           'batch_size': (16, 128),
           'regularization': (0.0001, 0.1),
           'hidden_layers': (1, 5),
           'hidden_neurons': (2,10)}

optimizer = BayesianOptimization(f=objective, pbounds=pbounds)
optimizer.maximize(init_points=10, n_iter=10)

best_hyperparameters = optimizer.max['params']
best_smape = -optimizer.max['target']

print("Best Hyperparameters:", best_hyperparameters)
print("Best SMAPE Score:", best_smape)

|   iter    |  target   | batch_... | hidden... | hidden... | learni... | regula... |
-------------------------------------------------------------------------------------
11/11 [==============================] - 0s 828us/step
| 1         | -24.15    | 68.74     | 1.331     | 3.294     | 0.06519   | 0.01557   |
11/11 [==============================] - 0s 996us/step
| 2         | -29.05    | 122.6     | 2.582     | 3.792     | 0.06996   | 0.08723   |
11/11 [==============================] - 0s 2ms/step
| 3         | -27.18    | 21.83     | 3.267     | 8.486     | 0.002729  | 0.06609   |
11/11 [==============================] - 0s 1ms/step
| 4         | -29.19    | 73.91     | 3.458     | 8.933     | 0.04349   | 0.07644   |
11/11 [==============================] - 0s 890us/step
| 5         | -23.98    | 75.38     | 1.923     | 9.856     | 0.0745    | 0.01713   |
11/11 [==============================] - 0s 2ms/step
| 6         | -32.98    | 50.66     | 4.205     | 2.405     | 0.02365   | 

KeyboardInterrupt: 

In [551]:
print(f"Regularization: {optimal_hyperparameters['regularization']}")
print(f"Learning Rate: {optimal_hyperparameters['learning_rate']}")
print(f"Batch Size: {optimal_hyperparameters['batch_size']}")

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(4, 1)),
], name="model")

for i in range(optimal_hyperparameters["hidden_layers"]):
    print(f"Hidden Neurons {optimal_hyperparameters['hidden_neurons'][i]} in Layer {i+1}.")
    model.add(tf.keras.layers.Dense(optimal_hyperparameters["hidden_neurons"][i], 
                                    activation="relu", 
                                    kernel_initializer=tf.initializers.HeNormal(), 
                                    kernel_regularizer=tf.keras.regularizers.l2(optimal_hyperparameters["regularization"])))
model.add(tf.keras.layers.Dense(1, activation="linear", 
                                kernel_initializer=tf.initializers.HeNormal(), 
                                kernel_regularizer=tf.keras.regularizers.l2(optimal_hyperparameters["regularization"])))

print()
model.compile(loss="mse",
                optimizer=tf.keras.optimizers.Adam(learning_rate=optimal_hyperparameters["learning_rate"]), 
                metrics=["mse", "mae"]) # Backpropagation

# Train the model on the full training dataset
model.fit(train_x, train_y, epochs=50, batch_size=optimal_hyperparameters["batch_size"], verbose=1, callbacks=[create_model_checkpoint(model_name=model.name)])

Regularization: 0.001
Learning Rate: 0.01
Batch Size: 32
Hidden Neurons 2 in Layer 1.
Hidden Neurons 7 in Layer 2.

Epoch 1/50
64/64 [==============================] - 1s 1ms/step - loss: 0.7296 - mse: 0.7151 - mae: 0.5659
Epoch 2/50
64/64 [==============================] - 0s 1ms/step - loss: 0.5776 - mse: 0.5656 - mae: 0.4831
Epoch 3/50
64/64 [==============================] - 0s 1ms/step - loss: 0.4935 - mse: 0.4827 - mae: 0.4201
Epoch 4/50
64/64 [==============================] - 0s 1ms/step - loss: 0.4504 - mse: 0.4403 - mae: 0.3729
Epoch 5/50
64/64 [==============================] - 0s 1ms/step - loss: 0.4327 - mse: 0.4234 - mae: 0.3613
Epoch 6/50
64/64 [==============================] - 0s 1ms/step - loss: 0.4216 - mse: 0.4130 - mae: 0.3547
Epoch 7/50
64/64 [==============================] - 0s 1ms/step - loss: 0.4218 - mse: 0.4138 - mae: 0.3624
Epoch 8/50
64/64 [==============================] - 0s 2ms/step - loss: 0.4139 - mse: 0.4064 - mae: 0.3530
Epoch 9/50
64/64 [==========

In [552]:
def autoregression(model, x, horizon=6):
    standardised_x = standardise(x)
    for i in range(horizon):
        forecast = model.predict(np.array([standardised_x[i:i+4]]))
        pred = np.array([tf.squeeze(forecast).numpy()])
        standardised_x = np.concatenate((standardised_x, pred))
    return de_standardise(standardised_x[-horizon:])

autoregression(model, np.array([4793.2, 5602, 5065, 5056]), 1)


1/1 [==============================] - 0s 63ms/step


array([4802.48985437])

In [510]:
data = (np.array([[940.66, 1084.86, 1244.98, 1445.02]]) - scaler.mean_) / scaler.scale_
print(data.shape)
#def make_preds(model, input_data):
#  forecast = model.predict(input_data)
#  preds = tf.squeeze(forecast)
#  return preds

#pred = make_preds(model, data)

[[-1.21687544 -1.13856182 -1.0407236  -0.92424603]]
1/1 [==============================] - 0s 53ms/step


In [511]:
np.array([[940.66, 1084.86, 1244.98, 1445.02]]).shape
#autoregression(model, test2_x[1], 1)
#standardised_x = np.array(standardise(test2_x[1]))

(1, 4)

In [251]:
array = np.array([[940.66, 1084.86, 1244.98, 1445.02]])
array[0][1:4]

array([1084.86, 1244.98, 1445.02])

[[-1.28740269 -1.21687544 -1.13856182 -1.0407236 ]]
1/1 [==============================] - 0s 22ms/step


In [512]:
inversed = de_standardise(np.array(pred))
inversed

array([1897.52436208])